# Fashion DNN 
- fashion_mnist 데이터 셋을 이용한 클래스 분류

# 1. 필수 라이브러리 임포트

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import torchvision
from torchvision import transforms
from torchvision.datasets import FashionMNIST

import numpy as np
from sklearn.metrics import accuracy_score

# 2. Dataset 다운로드

In [8]:
train_set = FashionMNIST("../0_data/", download=True, train=True, transform=transforms.ToTensor())
test_set = FashionMNIST("../0_data/", download=True, train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=True)

# 3. 모델 생성

In [9]:
class FashionDNN(nn.Module):
    def __init__(self):
        super(FashionDNN, self).__init__()
        self.sequnece = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784,256),
            nn.ReLU(inplace=True),
            nn.Linear(256,128),
            nn.ReLU(inplace=True),
            nn.Linear(128,10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.sequnece(x)
        return x

# 4. 모델 선언 및 컴파일

In [10]:
model = FashionDNN()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [15]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in (dataloader):
        inputs, targs = data

        optimizer.zero_grad()

        y_pred = model(inputs)
        loss = criterion(y_pred, targs)
        total_loss += loss

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)


def eval(model, dataloader, criterion):
    model.eval()

    y_pred_lst = []
    y_targ_lst = []
    loss = 0
    for idx, data in (enumerate(dataloader)):
        inputs, targs = data

        y_pred = model(inputs)

        loss += criterion(y_pred, targs)

        class_pred = torch.argmax(y_pred, dim=1)
        
        for i in class_pred:
            y_pred_lst.append(i.to("cpu").item())

        for i in targs:
            y_targ_lst.append(i.to("cpu").item())
    
    return y_targ_lst, y_pred_lst

In [17]:
for epoch in range(10):
    train_loss = train(model, train_loader, optimizer, criterion)
    
    print(f"Epoch : {epoch} \t loss : {train_loss:.6f}")

y_targ, y_pred = eval(model, test_loader, criterion)
print(f"정확도 : {accuracy_score(y_targ, y_pred):.2f}%")

Epoch : 0 	 loss : 1.638849
Epoch : 1 	 loss : 1.637327
Epoch : 2 	 loss : 1.636179
Epoch : 3 	 loss : 1.612612
Epoch : 4 	 loss : 1.579890
Epoch : 5 	 loss : 1.576927
Epoch : 6 	 loss : 1.571786
Epoch : 7 	 loss : 1.571096
Epoch : 8 	 loss : 1.572221
Epoch : 9 	 loss : 1.567185
정확도 : 0.86%
